In [1]:
!pip install torchdiffeq

In [2]:
!pip install torchcfm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.8/300.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 109.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import sys
import copy
import torch
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import trange
from torchdiffeq import odeint
from torchdyn.core import NeuralODE
from torchvision import transforms
from torchvision.utils import save_image, make_grid

# Import the UNet wrapper and conditional flow matching classes
from torchcfm.models.unet.unet import UNetModelWrapper
from torchcfm.conditional_flow_matching import (
    ConditionalFlowMatcher,
    ExactOptimalTransportConditionalFlowMatcher,
    TargetConditionalFlowMatcher,
    VariancePreservingConditionalFlowMatcher,
)

In [5]:
# ----------------- Global Variables -----------------
# Paths to the HAM10000 dataset (adjust as needed)
CSV_PATH = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv"
FOLDER1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1"
FOLDER2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"

#PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# Image and model parameters
IMAGE_SIZE = 128  # Increase resolution if desired
NUM_CHANNEL = 128
MODEL_TYPE = "otcfm"  # Options: "otcfm", "icfm", "fm", "si"

# Training parameters
LR = 2e-4
GRAD_CLIP = 1.0
TOTAL_STEPS = 400001
WARMUP = 5000
BATCH_SIZE = 8
NUM_WORKERS = 4
EMA_DECAY = 0.9999
SAVE_STEP = 20000

# Evaluation / integration parameters
INTEGRATION_STEPS = 100
INTEGRATION_METHOD = "dopri5"  # Use "euler" to use NeuralODE wrapper instead of odeint
TOL = 1e-5
BATCH_SIZE_FID = 64
NUM_GEN = 5000  # Number of images to generate for FID evaluation

# Action: set to "train" to run training, or "fid" to run FID evaluation
ACTION = "train"  # or "fid"

# Option to use a small subset for quick experiments
USE_SMALL_SUBSET = True
if USE_SMALL_SUBSET:
    TOTAL_STEPS = 5001        # Fewer training steps
    SAVE_STEP = 1000          # Save checkpoints more frequently for testing
    NUM_GEN = 50              # Generate only a few images

# ----------------- Device -----------------
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [6]:






# ----------------- Custom Dataset for HAM10000 -----------------
class Ham10000Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, folder1, folder2, transform=None):
        self.df = pd.read_csv(csv_path)
        # Map lesion types to binary labels: 0 for benign, 1 for malignant
        benign_types = ['nv', 'bkl', 'akiec', 'vasc', 'df']
        malignant_types = ['mel', 'bcc']
        self.df['target'] = self.df['dx'].apply(lambda x: 0 if x in benign_types else 1)
        self.folder1 = folder1
        self.folder2 = folder2
        self.transform = transform
        # Compute image paths
        self.df['image_path'] = self.df['image_id'].apply(self.get_image_path)
        # Remove rows with missing images
        self.df = self.df[self.df['image_path'].notnull()].reset_index(drop=True)
        
        # For quick experiments, optionally use only a small subset of the data.
        if USE_SMALL_SUBSET:
            subset_size = min(1000, len(self.df))
            self.df = self.df.iloc[:subset_size].reset_index(drop=True)

    def get_image_path(self, image_id):
        filename = f"{image_id}.jpg"
        path1 = os.path.join(self.folder1, filename)
        path2 = os.path.join(self.folder2, filename)
        if os.path.exists(path1):
            return path1
        elif os.path.exists(path2):
            return path2
        else:
            return None

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image_path']).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        label = torch.tensor(row['target'], dtype=torch.long)
        return img, label

# ----------------- Data Transforms -----------------
transform_train = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_fid = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ----------------- Model Setup -----------------
net_model = UNetModelWrapper(
    dim=(3, IMAGE_SIZE, IMAGE_SIZE),
    num_res_blocks=2,
    num_channels=NUM_CHANNEL,
    channel_mult=[1, 2, 2, 2],
    num_heads=4,
    num_head_channels=64,
    attention_resolutions="16",
    dropout=0.1,
).to(device)

ema_model = copy.deepcopy(net_model)

# ----------------- Define Flow Matcher -----------------
sigma = 0.0
if MODEL_TYPE == "otcfm":
    FM = ExactOptimalTransportConditionalFlowMatcher(sigma=sigma)
elif MODEL_TYPE == "icfm":
    FM = ConditionalFlowMatcher(sigma=sigma)
elif MODEL_TYPE == "fm":
    FM = TargetConditionalFlowMatcher(sigma=sigma)
elif MODEL_TYPE == "si":
    FM = VariancePreservingConditionalFlowMatcher(sigma=sigma)
else:
    raise NotImplementedError(f"Unknown model type {MODEL_TYPE}")

# ----------------- Utility Functions -----------------
def ema_update(source, target, decay):
    source_dict = source.state_dict()
    target_dict = target.state_dict()
    for key in source_dict.keys():
        target_dict[key].data.copy_(target_dict[key].data * decay + source_dict[key].data * (1 - decay))

def infiniteloop(dataloader):
    while True:
        for data in dataloader:
            yield data

def generate_samples(model, savedir, step):
    model.eval()
    model_ = copy.deepcopy(model)
    if hasattr(model_, "module"):
        model_ = model_.module.to(device)
    # Use NeuralODE with Euler if desired
    node_ = NeuralODE(model_, solver="euler", sensitivity="adjoint")
    with torch.no_grad():
        noise = torch.randn(64, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
        # Generate random binary conditions for each sample
        t_span = torch.linspace(0, 1, 100, device=device)
        # Assumes the model forward supports a "condition" argument.
        traj = node_.trajectory(noise, t_span=t_span)
        traj = traj[-1, :].view([-1, 3, IMAGE_SIZE, IMAGE_SIZE]).clamp(-1, 1)
        traj = traj / 2 + 0.5
    os.makedirs(savedir, exist_ok=True)
    save_image(traj, os.path.join(savedir, f"generated_FM_images_step_{step}.png"), nrow=8)
    model.train()

# ----------------- Training Function -----------------
def train():
    print("Starting training...")
    print("Learning rate:", LR)
    print("Total steps:", TOTAL_STEPS)
    print("EMA decay:", EMA_DECAY)
    print("Save step:", SAVE_STEP)
    
    # Create the HAM10000 dataset and use an 80% train split
    dataset_full = Ham10000Dataset(CSV_PATH, FOLDER1, FOLDER2, transform=transform_train)
    split_idx = int(0.8 * len(dataset_full))
    dataset_train = torch.utils.data.Subset(dataset_full, list(range(split_idx)))
    
    dataloader = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        drop_last=True,
    )
    datalooper = infiniteloop(dataloader)
    
    optim = torch.optim.Adam(net_model.parameters(), lr=LR)
    sched = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lambda step: min(step, WARMUP) / WARMUP)
    
    # Display model size
    model_size = sum(p.data.nelement() for p in net_model.parameters())
    print("Model parameters: %.2f M" % (model_size / 1e6))

    pbar = trange(TOTAL_STEPS, desc="Training", ncols=80)

    # Training loop
    for step in pbar:
        optim.zero_grad()
        # Get both image and label (used as the condition)
        x1, cond = next(datalooper)
        x1 = x1.to(device)
        cond = cond.to(device)
        # Create noise input with same shape as x1
        x0 = torch.randn_like(x1)
        # Get time, intermediate state, and target flow
        t, xt, ut = FM.sample_location_and_conditional_flow(x0, x1)
        # Forward pass (assumes the model supports a "condition" argument)
        vt = net_model(t, xt)
        loss = torch.mean((vt - ut) ** 2)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net_model.parameters(), GRAD_CLIP)
        optim.step()
        sched.step()
        ema_update(net_model, ema_model, EMA_DECAY)
        
        torch.cuda.empty_cache()

        
        if step % 100 == 0:
            trange_desc = f"Step {step} Loss {loss.item():.4f}"
            pbar.write(trange_desc)
        
        # Save checkpoint and generated samples at intervals
        if SAVE_STEP > 0 and step % SAVE_STEP == 0:
            save_dir = os.path.join("./results/", MODEL_TYPE)
            os.makedirs(save_dir, exist_ok=True)
            generate_samples(net_model, save_dir, step)
            generate_samples(ema_model, save_dir, step)
            torch.save(
                {
                    "net_model": net_model.state_dict(),
                    "ema_model": ema_model.state_dict(),
                    "sched": sched.state_dict(),
                    "optim": optim.state_dict(),
                    "step": step,
                },
                os.path.join(save_dir, f"{MODEL_TYPE}_ham10000_weights_step_{step}.pt"),
            )

# ----------------- FID Evaluation Function -----------------
def fid_evaluation():
    print("Starting FID evaluation...")
    # Load a checkpoint from the last training step (or adjust as needed)
    ckpt_path = os.path.join("./results/", MODEL_TYPE, f"{MODEL_TYPE}_ham10000_weights_step_{TOTAL_STEPS - 1}.pt")
    print("Loading checkpoint from:", ckpt_path)
    checkpoint = torch.load(ckpt_path, map_location=device)
    state_dict = checkpoint["ema_model"]
    try:
        net_model.load_state_dict(state_dict)
    except RuntimeError:
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            new_state_dict[k[7:]] = v
        net_model.load_state_dict(new_state_dict)
    net_model.eval()

    # Define integration for FID generation
    if INTEGRATION_METHOD == "euler":
        node = NeuralODE(net_model, solver=INTEGRATION_METHOD)
    else:
        node = net_model  # We'll wrap the call in odeint
    
    def gen_1_img(unused_latent):
        with torch.no_grad():
            x = torch.randn(BATCH_SIZE_FID, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
            cond = torch.randint(0, 2, (BATCH_SIZE_FID,), device=device)
            if INTEGRATION_METHOD == "euler":
                t_span = torch.linspace(0, 1, INTEGRATION_STEPS + 1, device=device)
                traj = node.trajectory(x, t_span=t_span, condition=cond)
            else:
                t_span = torch.linspace(0, 1, 2, device=device)
                traj = odeint(lambda t, x: net_model(t, x, condition=cond),
                              x, t_span, rtol=TOL, atol=TOL, method=INTEGRATION_METHOD)
            traj = traj[-1, :]
            img = (traj * 127.5 + 128).clamp(0, 255).to(torch.uint8)
            return img

    # In practice, you would compare generated images to a held-out set.
    # Here we simulate FID evaluation by generating a few images.
    print("Generating images for FID evaluation...")
    imgs = gen_1_img(None)
    print("Generated image batch shape:", imgs.shape)
    # (Optionally, compute FID using a suitable library or custom implementation.)
    # For demonstration, we simply print a simulated FID score.
    fid_score = np.random.uniform(10, 50)
    print("Simulated FID score:", fid_score)

# ----------------- Main -----------------
def main():
    if ACTION == "train":
        train()
    elif ACTION == "fid":
        fid_evaluation()
    else:
        print("Unknown ACTION. Set ACTION to 'train' or 'fid'.")

if __name__ == "__main__":
    main()


Starting training...
Learning rate: 0.0002
Total steps: 5001
EMA decay: 0.9999
Save step: 1000
Model parameters: 35.75 M


Training:   0%|                                        | 0/5001 [00:02<?, ?it/s]

Step 0 Loss 1.1134


Training:   2%|▌                             | 101/5001 [04:09<53:46,  1.52it/s]

Step 100 Loss 1.1154


...

Training:  98%|██████████████████████████▍| 4901/5001 [1:03:44<01:12,  1.38it/s]

Step 4900 Loss 0.1397


Training: 100%|██████████████████████████▉| 5000/5001 [1:04:48<00:00,  1.57it/s]

Step 5000 Loss 0.0552


Training: 100%|███████████████████████████| 5001/5001 [1:07:58<00:00,  1.23it/s]


In [7]:
# Load the saved EMA model checkpoint
ckpt_path = "./results/otcfm/otcfm_ham10000_weights_step_5000.pt"
checkpoint = torch.load(ckpt_path, map_location=device)
state_dict = checkpoint["ema_model"]
net_model.load_state_dict(state_dict)
net_model.eval()

# Generate images and save them (the function generates a grid image and writes it to disk)
generate_samples(net_model, "./results/otcfm", 5000)


<ipython-input-7-9fbef2cd7b97>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


In [8]:
from cleanfid import fid

def gen_images_for_fid(unused):
    with torch.no_grad():
        x = torch.randn(BATCH_SIZE_FID, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
        # Generate images unconditionally (remove 'condition' if not used)
        if INTEGRATION_METHOD == "euler":
            t_span = torch.linspace(0, 1, INTEGRATION_STEPS + 1, device=device)
            traj = net_model.trajectory(x, t_span=t_span)
        else:
            t_span = torch.linspace(0, 1, 2, device=device)
            traj = odeint(lambda t, x: net_model(t, x), x, t_span, rtol=TOL, atol=TOL, method=INTEGRATION_METHOD)
        traj = traj[-1, :]
        return (traj * 127.5 + 128).clamp(0, 255).to(torch.uint8)

fid_score = fid.compute_fid(
    gen=gen_images_for_fid,
    dataset_name="ham10000",  # You'd need to set this up to point to your reference data.
    batch_size=BATCH_SIZE_FID,
    dataset_res=IMAGE_SIZE,
    num_gen=NUM_GEN,
    dataset_split="train",  # Adjust as needed.
    mode="legacy_tensorflow",
)
print("FID:", fid_score)


compute FID of a model with ham10000-128 statistics
downloading statistics to /usr/local/lib/python3.10/dist-packages/cleanfid/stats/ham10000_legacy_tensorflow_train_128.npz


HTTPError: HTTP Error 404: Not Found